In [1]:
# Libraries for data manipulation and linear algebra
import numpy as np 
import pandas as pd 

# For file paths
from pathlib import Path  

# Tensorflow and Keras for Neural Network
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

# Scikit-learn libraries for preprocessing, model evaluation, and text vectorization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import mean_squared_error

# Libraries for text processing
import re
import spacy
from spacy.lang.en import English 
from spacy.lang.en.stop_words import STOP_WORDS

# Visualization
import matplotlib.pyplot as plt

%matplotlib inline

# Display input files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/summary-test/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


Load and Preprocess Dataset

In [2]:
# Load the dataset
file_path = '/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv'
se = pd.read_csv(file_path)

# Drop rows with missing 'wording' and 'content'
se.dropna(subset=['wording', 'content'], inplace=True)

# Fill missing 'text' values
se['text'].fillna('', inplace=True)

# Text preprocessing function
def preprocess_text(text):
    text = text.replace('\n', ' ').lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = ' '.join([word for word in text.split() if word.isalpha() and word not in STOP_WORDS])
    return ' '.join(text.split())

# Apply the preprocessing function to texts
se['PreprocessedText'] = se['text'].apply(preprocess_text)


Vectorization of Text Data

In [3]:
# Vectorize preprocessed texts
vectorizer = CountVectorizer(binary=True, max_features=5000)
X = vectorizer.fit_transform(se['PreprocessedText'])

# Split data into training and testing sets
y = se[['wording', 'content']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Data Preparation for Neural Network

In [4]:
# Convert sparse matrix to dense format
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_dense)
X_test_scaled = scaler.transform(X_test_dense)


Neural Network Model Building and Training

In [5]:
# Define model architecture
hidden_nodes_l1 = (X_train_scaled.shape[1] + 1) // 2
hidden_nodes_l2 = (hidden_nodes_l1 + 1) // 2
hidden_nodes_l3 = (hidden_nodes_l2 + 1) // 2
hidden_nodes_l4 = (hidden_nodes_l3 + 1) // 2

model = Sequential([
    Dense(hidden_nodes_l1, input_dim=X_train_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.25),
    Dense(hidden_nodes_l2, activation='tanh', kernel_regularizer=l2(0.001)),
    Dropout(0.25),
    Dense(hidden_nodes_l3, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.25),
    Dense(hidden_nodes_l4, activation='tanh', kernel_regularizer=l2(0.001)),
    # Output layer with two nodes for "wording" and "content"
    Dense(2, activation='sigmoid')  
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
144/144 [==============================] - 28s 185ms/step - loss: 2.4744 - mse: 0.9974 - val_loss: 1.4149 - val_mse: 0.8141
Epoch 2/50
144/144 [==============================] - 25s 175ms/step - loss: 1.4066 - mse: 0.8445 - val_loss: 1.3769 - val_mse: 0.8182
Epoch 3/50
144/144 [==============================] - 25s 171ms/step - loss: 1.3384 - mse: 0.8251 - val_loss: 1.3330 - val_mse: 0.8136
Epoch 4/50
144/144 [==============================] - 24s 170ms/step - loss: 1.4115 - mse: 0.8449 - val_loss: 1.4020 - val_mse: 0.8070
Epoch 5/50
144/144 [==============================] - 25s 173ms/step - loss: 1.3551 - mse: 0.8108 - val_loss: 1.3172 - val_mse: 0.7928
Epoch 6/50
144/144 [==============================] - 24s 167ms/step - loss: 1.2881 - mse: 0.7852 - val_loss: 1.2422 - val_mse: 0.7763
Epoch 7/50
144/144 [==============================] - 24s 169ms/step - loss: 1.1907 - mse: 0.7639 - val_loss: 1.1824 - val_mse: 0.7564
Epoch 8/50
144/144 [==============================] - 2

Evaluation

In [6]:
# Predict on test data
y_pred = model.predict(X_test_scaled)

rmse_wording = mean_squared_error(y_test[:, 0], y_pred[:, 0], squared=False)
rmse_content = mean_squared_error(y_test[:, 1], y_pred[:, 1], squared=False)

print(f"RMSE for wording: {rmse_wording}")
print(f"RMSE for content: {rmse_content}")


45/45 [==============================] - 1s 18ms/step
RMSE for wording: 0.9183793035035119
RMSE for content: 0.863933074639097


Prepare Submission for Competition

In [7]:
# Load test dataset for competition
test_df = pd.read_csv(Path("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv"))

# Preprocess and vectorize the test data
test_df['PreprocessedText'] = test_df['text'].apply(preprocess_text)
X_test_competition = vectorizer.transform(test_df['PreprocessedText'])

# Predict using the trained model
predictions = model.predict(X_test_competition)

# Prepare submission DataFrame
submission_df = pd.DataFrame({
    'student_id': test_df['student_id'],
    'content': predictions[:, 1],
    'wording': predictions[:, 0]
})

# Clip values to be within expected bounds
submission_df['content'] = np.clip(submission_df['content'], -2, 5)
submission_df['wording'] = np.clip(submission_df['wording'], -2, 5)

# Save submission to a CSV
submission_df.to_csv("submission.csv", index=False)


1/1 [==============================] - 0s 125ms/step
